## Test File for _project_and_slide_moving functionality

In [ ]:
from io import StringIO
import sys
import numpy as np
import matplotlib.pyplot as plt
from planktos import swarm
from planktos.geom import seg_intersect_2D_multilinear_poly as seg_intersect

# Capture print outputs
class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

# Helper to set t_crit
def set_t_crit():
    if t_rot is not None and t_edge is not None:
        return max(t_rot, t_edge)
    elif t_rot is not None:
        return t_rot
    elif t_edge is not None:
        return t_edge
    else:
        return None

# Create a function that will plot the geometry
def plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit=None, newstartpt=None, newendpt=None):
    # unpack intersection
    x = intersection[0]    # (x,y) coordinates of intersection
    t_I = intersection[1]  # btwn 0 & 1, fraction of movement traveled so far
    Q0 = intersection[2]   # edge of mesh element at time of intersection
    Q1 = intersection[3]   # edge of mesh element at time of intersection
    idx = intersection[4]  # index into close_mesh_start/end above
    
    # plot start and end position of mesh
    plt.plot(start_mesh[:,:,0].flatten(), start_mesh[:,:,1].flatten(), 'o-', color='lightgrey', label='startmesh')
    plt.plot(end_mesh[:,:,0].flatten(), end_mesh[:,:,1].flatten(), 'o-', color='black', label='endmesh')
    # plot original start and end position of agent
    plt.plot((startpt[0], endpt[0]), (startpt[1], endpt[1]), 'o-', label='orig trajectory')
    # plot arrow showing direction of agent
    xpos = (startpt[0]*0.8+endpt[0]*0.2); ypos = (startpt[1]*0.8+endpt[1]*0.2)
    xdir = endpt[0]-startpt[0]; ydir = endpt[1]-startpt[1]
    plt.annotate("", xytext=(xpos, ypos), xy=(xpos+0.001*xdir, ypos+0.001*ydir), arrowprops=dict(arrowstyle="->", color="k"), size=20)
    # plot element that was intersected at time of intersection
    plt.plot((Q0[0], Q1[0]), (Q0[1], Q1[1]), label='element at intersection')

    # if a critical time is given, plot the relevant mesh element at that time as well
    if t_crit is not None:
        interp_mesh_el = start_mesh[idx]*(1-t_crit) + end_mesh[idx]*t_crit
        plt.plot((interp_mesh_el[0,0],interp_mesh_el[1,0]),(interp_mesh_el[0,1],interp_mesh_el[1,1]), label='element at t_crit')
    # if new start and end points are given, plot the new trajectory off the element
    if newstartpt is not None:
        plt.plot((newstartpt[0], newendpt[0]), (newstartpt[1], newendpt[1]), 'o-', color='goldenrod', label='new trajectory')
    
    # plot point of intersection
    plt.plot(x[0], x[1], 'x', color='black')
    # plot final point
    plt.plot(finalendpt[0], finalendpt[1], 's', color='black')

In [ ]:
# test for moving boundary intersecting a stationary agent
# in-line
# start_mesh = np.array([
#     [[0.8,0], [0.8,1]],
#     [[0.8,1], [0.8,2]]
# ])
# end_mesh = np.array([
#     [[1.2,0], [1.2,1]],
#     [[1.2,1], [1.2,2]]
# ])
# startpt = np.array([1,0.5])
# endpt = np.array([1,0.5])

# slanted translation - inertial forces and drag forces on the agent are neglected,
#    but the force applied perp to the element slides the agent a bit.
# start_mesh = np.array([
#     [[0.3,0], [1,2]]
# ])
# end_mesh = np.array([
#     [[1,0], [1.7,2]]
# ])

# rotating
start_mesh = np.array([
    [[1,0], [0.8,2]]
])
end_mesh = np.array([
    [[1,0], [3,1.5]]
])

startpt = np.array([1,1])
endpt = np.array([1,1])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
print(intersection[0])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)

In [ ]:
# penetration debug in rubberband where movement is in one direction but mesh slides by in the other
start_mesh = np.array([[[0.29605039954186, 0.39581149816513],
        [0.3109315931797 , 0.38228872418404]],
       [[0.3109315931797 , 0.38228872418404],
        [0.32728180289269, 0.37046629190445]],
       [[0.32728180289269, 0.37046629190445],
        [0.34523025155067, 0.35993620753288]],
       [[0.34523025155067, 0.35993620753288],
        [0.36465215682983, 0.35081824660301]]]
)
end_mesh = np.array([[[0.30339360237122, 0.39260469377041],
        [0.31745845079422, 0.37880085408688]],
       [[0.31745845079422, 0.37880085408688],
        [0.33305545151234, 0.36663369834423]],
       [[0.33305545151234, 0.36663369834423],
        [0.35021229088306, 0.35584428906441]],
       [[0.35021229088306, 0.35584428906441],
        [0.36883965134621, 0.34648998081684]]]
)
startpt = np.array([0.3286098993525 , 0.36968712049628])
endpt = np.array([0.33095932138719, 0.36587515123914])
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
np.set_printoptions(precision=14)
print(intersection)
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
plt.xlim(finalendpt[0]-0.01,finalendpt[0]+0.02)
plt.ylim(finalendpt[1]-0.005,finalendpt[1]+0.005)
print(finalendpt)

In [ ]:
# single mesh element, rotating around
start_mesh = np.array([
    [[0.5,0], [1.5,0.4]]
])
end_mesh = np.array([
    [[0.5,0], [0.1,0.9]]
])
startpt = np.array([0,0.4])
endpt = np.array([1,0.4])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)

In [ ]:
# convex mesh elements, moving, edge reached when adj element acute
start_mesh = np.array([
    [[0, 0], [0.4, 0.4]],
    [[0.4, 0.4], [0.6, 0.5]]
])
end_mesh = np.array([
    [[0, 0], [0.4, 0.4]],
    [[0.4, 0.4], [0.05, 0.4]]
])
startpt = np.array([0,0.2])
endpt = np.array([1.5,0.2])
# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)

In [ ]:
import pdb; pdb.pm()

In [ ]:
# convex mesh elements, moving, edge reached when adj element is perp
start_mesh = np.array([
    [[0, 0], [0.4, 0.4]],
    [[0.4, 0.4], [0.6, 0.5]]
])
end_mesh = np.array([
    [[0, 0], [0.4, 0.4]],
    [[0.4, 0.4], [0.2, 0.5]]
])
startpt = np.array([0,0.2])
endpt = np.array([1.5,0.2])
# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)

In [ ]:
# convex mesh elements, moving
start_mesh = np.array([
    [[0.8, 0.2], [0.8, 0.9]],
    [[0.8, 0.9], [0.9, 1.2]]
])
end_mesh = np.array([
    [[0.8, 0.2], [0.8, 0.9]],
    [[0.8, 0.9], [1, 1.2]]
])
startpt = np.array([0.5,0.5])
endpt = np.array([1.5,1.25])
# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)

In [ ]:
# single mesh element, rotating away, deforming
start_mesh = np.array([
    [[0,0.25], [0.8,1]]
])
end_mesh = np.array([
    [[0,0.25], [1.4,0.4]]
])
startpt = np.array([0,0.3])
endpt = np.array([1,0.3])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)


In [ ]:
import pdb; pdb.pm()

In [ ]:
# single mesh element angled, slide off, deforming
start_mesh = np.array([
    [[0.5,0.25], [1,0.75]]
])
end_mesh = np.array([
    [[0.6,0.35], [0.9,0.65]]
])
startpt = np.array([0,0.5])
endpt = np.array([1.5,0.5])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)

In [ ]:
# in-line mesh elements, angled hit, overtaking
start_mesh = np.array([
    [[0.5,0], [0.5,1]],
    [[0.5,1], [0.5,2]]
])
end_mesh = np.array([
    [[1.5,0], [1.5,1]],
    [[1.5,1], [1.5,2]]
])

startpt = np.array([0.75,0.5])
endpt = np.array([1.25,1.25])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
print(t_crit)
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt)
print(finalendpt)

In [ ]:
# single mesh element, slide off, moving, deforming
start_mesh = np.array([
    [[0.8,0], [0.8,0.8]]
])
end_mesh = np.array([
    [[1.2,0], [1.2,1.1]]
])
startpt = np.array([0.5,0.5])
endpt = np.array([1.5,1.25])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(finalendpt)

In [ ]:
# deforming mesh element
start_mesh = np.array([
    [[0.8,0], [0.8,1]]
])
end_mesh = np.array([
    [[1.2,0], [1.2,2]]
])
startpt = np.array([0.5,0.5])
endpt = np.array([1.5,1.5])
# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(intersection[0])
print(finalendpt)

In [ ]:
# single mesh element, moving, slide off opposite side
start_mesh = np.array([
    [[0.8,0.4], [0.8,0.8]]
])
end_mesh = np.array([
    [[1,1.1], [1,1.2]]
])
startpt = np.array([0.5,0.7])
endpt = np.array([1.2,1])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None; newstartpt = None; newendpt = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
plt.legend()
print(finalendpt)

In [ ]:
# in-line mesh elements, angled approach, moving and non-moving
start_mesh = np.array([
    [[0.8,0], [0.8,1]],
    [[0.8,1], [0.8,2]]
])
end_mesh = np.array([
    [[1.2,0], [1.2,1]],
    [[1.2,1], [1.2,2]]
])
startpt = np.array([0.5,0.5])
endpt = np.array([1.5,1.25])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 10, 10)
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt)
print(finalendpt)